In [37]:
#TEST
#Call another function within a class

class TestClass:
    
    def print_x3(self, string):
        return string*3
        
    def print_x6(self, string):
        return self.print_x3(string) * 2
        
test_ins = TestClass()
type(test_ins)

test_ins.print_x6("good")

'goodgoodgoodgoodgoodgood'

In [53]:
import MeCab
import pandas as pd
import numpy as np
import re

class MecabParser:

    def __init__(self, file_in):
        self.file_in = file_in
        self.file_out = self.file_in + ".mecab"
        
    def initial_parser(self):
        file_in = self.file_in
        file_out = self.file_out
        tagger = MeCab.Tagger("-Ochasen")
        
        with open(file_in, encoding="utf-8") as input_file:
            with open(file_out, mode="w", encoding="utf-8") as output_file:
                parsed_result = tagger.parse(input_file.read())
                output_file.write(parsed_result)

        #CREATE RAW TABLE
        raw_table = pd.read_table(file_out)
        raw_table.columns = range(0, raw_table.shape[1]) #Name column headers other the first word row becomes the column headers
        self.raw_table = raw_table
        return self.raw_table
    
    def clean_parser(self):
        #CREATE NEW TABLE
        new_sorted_data = self.raw_table[[0, 1, 3]].copy() #Only take Columns 0, 1, and 3
        new_sorted_data.columns = ["単語", "フリガナ", "品詞"]
        
        #Remove Common Words Such as Fucntional Words
        functional_word_index = []
        row_count = new_sorted_data.shape[0]
        for i in np.arange(row_count):
            pos_str =str(new_sorted_data.loc[i, "品詞"])
            word_str = str(new_sorted_data.loc[i, "単語"])
            re_kanji = re.compile(r'^[\u4E00-\u9FD0]+$')
            status_kanji = re_kanji.fullmatch(word_str)
            if pos_str[:3] in ["助詞", "記号"] or \
                status_kanji == None:
                functional_word_index.append(i)        
                
        word_table = new_sorted_data.drop(functional_word_index)
        
        #Sort and Drop Duplicates
        word_table = word_table.sort_values(by = "単語")
        word_table = word_table.drop_duplicates() #DataFrame.drop_duplicates(self, subset=None, keep='first', inplace=False)[source]
        word_table.index = np.arange(word_table.shape[0])
        word_table.style.set_properties(**{'text-align': 'right'})
        self.word_table = word_table
        
        return self.word_table
    
    def hiragana_parser(self):
        hira_table = self.word_table[:]
        hira_table.insert(1, "ふりがな", hira_table[:].iloc[:, 1])

        row_count = len(hira_table)
        for i in range(row_count):
            pronunciation = hira_table.iloc[i, 1]
            hira_table.iloc[i, 1] = self.kata_to_hira(pronunciation)

        hira_table = hira_table.drop(["フリガナ"], axis=1)
        
        return hira_table
    
    
    def kata_to_hira(self, word):
        str_hira = "あいうえおぁぃぅぇぉかきくけこがぎぐげごさしすせそざじずぜぞたちつてとっだぢづでどなにぬねのはひふへほばびぶべぼぱぴぷぺぽまみむめもやゆよゃゅょらりるれろわをん"
        str_kata = "アイウエオァィゥェォカキクケコガギグゲゴサシスセソザジズゼゾタチツテトッダヂヅデドナニヌネノハヒフヘホバビブベボパピプペポマミムメモヤユヨャュョラリルレロワヲン"

        list_hira = []
        for i in str_hira:
            list_hira.append(i)

        list_kata = []
        for i in str_kata:
            list_kata.append(i)

        dic_kata_to_hira = {}
        for i in np.arange(len(list_kata)):
                dic_kata_to_hira.update({list_kata[i]:list_hira[i]})
        new_word = ""

        for index, char in enumerate(word):
            if char in dic_kata_to_hira.keys():
                new_word += dic_kata_to_hira[char]
            else:
                new_word += char

        return new_word
    

In [54]:
# test_str = "私はエンジニアです。東京に住んでおります。自然言語処理は大好きです。よろしくお願いいたします！"
# df_test_str = pd.DataFrame([test_str])
# df_test_str

# df_test_str.to_csv("./data/test_str.txt", index=False, header=False)

In [55]:
file_path = "./data/test_str.txt"
parser = MecabParser(file_path)

parser.initial_parser()

,0,1,2,3,4,5
0,は,ハ,は,助詞-係助詞,NaN,NaN
1,エンジニア,エンジニア,エンジニア,名詞-一般,NaN,NaN
2,です,デス,です,助動詞,特殊・デス,基本形
3,。,。,。,記号-句点,NaN,NaN
4,東京,トウキョウ,東京,名詞-固有名詞-地域-一般,NaN,NaN
5,に,ニ,に,助詞-格助詞-一般,NaN,NaN
6,住ん,スン,住む,動詞-自立,五段・マ行,連用タ接続
7,で,デ,で,助詞-接続助詞,NaN,NaN
8,おり,オリ,おる,動詞-非自立,五段・ラ行,連用形
9,ます,マス,ます,助動詞,特殊・マス,基本形


In [56]:
parser.clean_parser()

,単語,フリガナ,品詞
0,処理,ショリ,名詞-サ変接続
1,東京,トウキョウ,名詞-固有名詞-地域-一般
2,自然,シゼン,名詞-形容動詞語幹
3,言語,ゲンゴ,名詞-一般


In [57]:
kata_table = parser.hiragana_parser()

type(kata_table)
#pandas.core.frame.DataFrame
kata_table

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-53-b733e256ab87>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hira_table.iloc[i, 1] = self.kata_to_hira(pronunciation)


,単語,ふりがな,品詞
0,処理,しょり,名詞-サ変接続
1,東京,とうきょう,名詞-固有名詞-地域-一般
2,自然,しぜん,名詞-形容動詞語幹
3,言語,げんご,名詞-一般


In [ ]:
def get_jp_stress(vocab_list):
    import re
    import requests
    import time

    from bs4 import BeautifulSoup
    from random import random
    from selenium import webdriver

    driver = webdriver.Chrome()
    just_a_sec = random() 
    search_string = vocab_list
    
    stress_list =[]
    stress_list_with_word =[]
    for word in vocab_list:
        if len(stress_list) <= 300:
            try:    
                search_site = "https://www.weblio.jp/content/"
                search_site = search_site + word

                driver.get(search_site)
                time.sleep(just_a_sec * 1)

                search_box = driver.find_element_by_class_name("formBox")

                result_furigana = driver.find_element_by_xpath('//*[@id="cont"]/div[3]/div/div[1]/h2/b')
                result_stress = driver.find_element_by_xpath('//*[@id="cont"]/div[3]/div/div[1]/h2/span')

                furigana = result_furigana.text
                word_stress = result_stress.text
                
                stress_list.append(word_stress)
                #stress_list_with_word.append(word + ": " + furigana + ": " + word_stress)
                
            except:
                stress_list.append("[na]")
                #stress_list_with_word.append(word + ": " + "[na]")
    
    return stress_list#, stress_list_with_word

In [ ]:
# #CREATE A KATAKANA-HIRAGANA DICTIONARY

# str_hira = "あいうえおぁぃぅぇぉかきくけこがぎぐげごさしすせそざじずぜぞたちつてとっだぢづでどなにぬねのはひふへほばびぶべぼぱぴぷぺぽまみむめもやゆよゃゅょらりるれろわをん"
# str_kata = "アイウエオァィゥェォカキクケコガギグゲゴサシスセソザジズゼゾタチツテトッダヂヅデドナニヌネノハヒフヘホバビブベボパピプペポマミムメモヤユヨャュョラリルレロワヲン"

# list_hira = []
# for i in str_hira:
#     list_hira.append(i)
    
# list_kata = []
# for i in str_kata:
#     list_kata.append(i)

# dic_kata_to_hira = {}
# for i in np.arange(len(list_kata)):
#         dic_kata_to_hira.update({list_kata[i]:list_hira[i]})

        
# def kata_to_hira(word):
#     new_word = ""

#     for index, char in enumerate(word):
#         if char in dic_kata_to_hira.keys():
#             new_word += dic_kata_to_hira[char]
#         else:
#             new_word += char

#     return new_word

In [ ]:
result = mecab_parse("flu_detail.txt")
result

In [ ]:
vocab_list = result.iloc[:, 0]

list_of_words = []
count = 0
for i in vocab_list:
    list_of_words.append(i)
    
list_of_words

In [ ]:
#GET STRESS ACCENTS

dic_result = get_jp_stress(list_of_words)
dic_result

In [ ]:
dic_result_completed = dic_result[:]
for i in np.arange(len(dic_result), len(list_of_words)):
        dic_result_completed.append("na")
        
dic_result_completed

new_result = result[:]
new_result.insert(3, "アクセント", dic_result_completed)
new_result

In [ ]:
series_furi = new_result.iloc[:, 2]

for index, element in enumerate(series_furi):
    series_furi[index] = kata_to_hira(element)
series_furi

new_result.iloc[:20, [0,2,3,4]]

In [ ]:
revised = new_result.iloc[:, [0,2,3,4]]
#revised = new_result.iloc[:, [0,2,4]]
revised

#revised.to_csv("revised.csv" , encoding="shift_jis") #invalid character!
#revised.to_csv("revised.csv" , encoding="utf8") #characters are garbled!

In [ ]:
numbers = []
for i in np.arange(10):
    numbers += str(i) # numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    
def chk_for_number(word):
    result = False
    for i in word:
        if i in numbers:
            result = True
    return result

In [ ]:
revised2 = revised[:]

series_stress = revised2.iloc[:, 2]
series_stress

for index, element in enumerate(series_stress):
    if chk_for_number(element) == False:
        series_stress[index] = "[na]"

revised2

revised2.to_csv("revised2.csv" , encoding="shift_jis") 
revised2.to_csv("revised.csv" , encoding="utf8") 